## Running simulations using Popy

In the previous part of this introduction, we have learned how to use Popy to build, evaluate and export static networks.
In the following part of this introduction, we will learn how to use Popy to directly run a simulation based on the defined network.
To perform simulations, Popy relies on [AgentPy](https://agentpy.readthedocs.io/en/latest/#) - a general ABM framework.
We recommend you to familiarize yourself with AgentPy first, as only some aspects of AgentPy are covered in this introduction.

In the following, we programme a simple infection simulation as an example of how to use Popy to run simulations.

### DataFaker

In the previous example, we used example data to create the population of agents.
In this example, we also use *fake* data, but this time we use a data generator provided by Popy to generate the data.
The DataFaker creates a data set of any size, which is similar to a classic survey data set.
`popy.data_fakers.soep_faker` creates data similiar to the [German Socio-Economic Panel](https://www.diw.de/en/diw_01.c.678568.en/research_data_center_soep.html).

We start by creating a dataset of 1000 rows:

In [ ]:
from popy.data_fakers.soep import soep_faker

df_soep = soep_faker.soep(size=1000)
df_soep.head()

### A first Model

The object class `Model` is a key component of implementing simulation models using Popy (or AgentPy).
The class `Model` integrates all elements of the simulation and determines the simulation sequences.
Every simulation model which relies on Popy must have exactly one instance of `Model`.
(In fact, even the generation of static networks needs an instance of `Model`, which is created for you under the hood, to keep the code as simple as possible.)

`Model` has four core methods that structure the processes of the simulation model:

1. `Model.setup()`
2. `Model.step()`
3. `Model.update()`
4. `Model.end()`

Please refer to the [AgentPy documentation](https://agentpy.readthedocs.io/en/latest/reference_model.html) for more detailed information on those methods.

In addition to an *empty* `Agent` class and an *empty* `Location` class, we define our first own `Model` class below.
First we define the method `setup()`, which is executed according to the AgentPy logic at the beginning of the simulation.
Within the `setup()` method, we do nothing more than create a `Creator` as usual and then create agents and locations.
This time, however, we save everything as attributes of the `InfectionModel`.
It is also important to pass the `InfectionModel` to the `Creator` in the form of `self` via the `model` parameter.
This ensures that all agents and locations created via this `Creator` are assigned to the `InfectionModel` and vice versa.
This allows agents to access their environment during the simulation, for example.

In [ ]:
import random

import popy

class Agent(popy.Agent):
    pass


class Home(popy.Location):
    pass


class InfectionModel(popy.Model):
    def setup(self):
        self.creator = popy.Creator(model=self)
        self.agents = self.creator.create_agents(df=df_soep, agent_class=Agent)
        self.locations = self.creator.create_locations(
            agents=self.agents, location_classes=[Home],
        )

Now, we execute the simulation model.
Again, please refer to the [AgentPy documentation](https://agentpy.readthedocs.io/en/latest/overview.html#running-a-simulation) if needed.

In [ ]:
parameters = {
    "steps": 50,
}
model = InfectionModel(parameters=parameters)
results = model.run()

The simulation has now run, but practically nothing has happened except that the agents and locations have been created.
The `model` object now has the `agents` attribute, for example:

In [ ]:
model.agents

In [ ]:
model.locations

Now we add some code to make the model a working infection simulation.
We add the attributes `infection_status` and `days_since_infection` as well as the methods `infect` and `update_infection_status` to the agent.

What is relevant here is how the agent accesses other agents with which it is in contact.
This is done using the `Agent.neighbors()` method.
On the other hand, it is relevant how the agent determines *how much* contact it has or has had with the other agents.
This is done using the method `Agent.contact_weight()`.
Both methods include all locations that an agent visits.

In [ ]:
class Agent(popy.Agent):
    def __init__(self, model, *args, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.infection_status = "susceptible"
        self.days_since_infection = 0

    def infect(self):
        if self.infection_status in ["infectious", "symptomatic"]:
            for agent_v in self.neighbors():
                if agent_v.infection_status == "susceptible":
                    infection_probability = 0.01 * self.contact_weight(agent_v)

                    if random.random() < infection_probability:
                        agent_v.infection_status = "exposed"


    def update_infection_status(self):
        if self.infection_status in ["exposed", "infectious", "symptomatic"]:
            self.days_since_infection += 1

            if 2 <= self.days_since_infection <= 5:
                self.infection_status = "infectious"

            elif 6 <= self.days_since_infection <= 10:
                self.infection_status = "symptomatic"

            elif self.days_since_infection >= 11:
                self.infection_status = "recovered"

Next, we define 4 locations.
A home `Home`, to which all agents of a household are assigned, a workplace `Work`, which groups the agents according to their occupational sectors, and a school `School`, which contains age-specific classes.
To ensure that each agent has contact with at least two other agents, we also define the location `Ring`, which connects the entire population along a ring using the predefined Popy class `RingLocation`.

In [ ]:
class Home(popy.Location):
    def split(self, agent):
        return agent.hid

    def weight(self, agent):
        return 12

class Work(popy.Location):
    def setup(self):
        self.n_agents = 10

    def filter(self, agent):
        return True if agent.work_hours_day > 0 and agent.nace2_division > 0 else False

    def split(self, agent):
        return agent.nace2_division

    def weight(self, agent):
        return agent.work_hours_day

class School(popy.Location):
    def setup(self):
        self.n_agents = 25

    def filter(self, agent):
        return True if 6 <= agent.age <= 18 else False
    
    def split(self, agent):
        return agent.age

    def weight(self, agent):
        return 6

class Ring(popy.RingLocation):
    pass

We extend the `InfectionModel` so that 10 random agents are infected at the start of the simulation (`setup()`).
In `step()` we define what should happen in each individual time step of the simulation.
First, the method `update_infection_status()` is to be executed for each agent and then the method `infect()` for each agent.
If you are wondering about the strange syntax, have a look [here](https://agentpy.readthedocs.io/en/latest/overview.html#agent-sequences).
The method `update()` is also executed in each time step, but always after the method `step()`.
In `update()` we collect the number of agents ever infected per time step in order to visualize it after the simulation.

In [ ]:
class InfectionModel(popy.Model):
    def setup(self):
        self.creator = popy.Creator(model=self)
        self.agents = self.creator.create_agents(df=df_soep, agent_class=Agent)
        self.locations = self.creator.create_locations(
            agents=self.agents, 
            location_classes=[
                Home, 
                Work, 
                School, 
                Ring,
                ])

        for agent in random.choices(self.agents, k=10):
            agent.infection_status = "exposed"
    
    def step(self):
        self.agents.update_infection_status()
        self.agents.infect()

    def update(self):
        self.record(
            "cumulative_infections",
            sum([1 for agent in self.agents if agent.infection_status != "susceptible"]),
        )

Let's execute the model:

In [ ]:
parameters = {
    "steps": 50,
}
model = InfectionModel(parameters=parameters)
results = model.run()

Below, you can see the number of infections per time step.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results.variables.InfectionModel.cumulative_infections)
plt.grid()

Just because we can, let's have look at the network:

In [ ]:
model.creator.plot_bipartite_network()

In [ ]:
model.creator.plot_agent_network()

### Dynamic networks

Up to now, we have had purely static networks that have not changed once they have been created.
In Popy, there are two ways to change the networks during the simulation.
Firstly, agents can leave and join locations using the methods `Location.remove_agent()` and `Location.add_agent()`.
If a complete *move* of the agent from one location to another is not necessary, the connection weights between the agent and the location can also simply be varied.
If possible, this method is always preferable.
In the following, we will have a look at this method to create dynamic networks.

We now want to add another property to the infection simulation: Agents only visit the school or workplace if they are not currently symptomatically ill.

In order to exclude agents who are symptomatically ill from schools and workplaces, we adapt the `weight()` method in each case.
By default, the `weight()` method is executed repeatedly in each time step for each location so that the weights are always up to date
(as this can affect the performance of the simulation, you can set the attribute `Location.static_weight = True` in the `setup()` method to deactivate this function).

In [ ]:
class Home(popy.Location):
    def split(self, agent):
        return agent.hid

    def weight(self, agent):
        if agent.infection_status == "symptomatic":
            return 3
        else:
            return 12

    def project_weights(self, agent1, agent2):
        return min(self.get_weight(agent1), self.get_weight(agent2))

class Work(popy.Location):
    def setup(self):
        self.n_agents = 10

    def filter(self, agent):
        return True if agent.work_hours_day > 0 and agent.nace2_division > 0 else False

    def split(self, agent):
        return agent.nace2_division

    def weight(self, agent):
        if agent.infection_status == "symptomatic":
            return 0
        else:
            return agent.work_hours_day
    
    def project_weights(self, agent1, agent2):
        return min(self.get_weight(agent1), self.get_weight(agent2))

class School(popy.Location):
    def setup(self):
        self.n_agents = 25

    def filter(self, agent):
        return True if 6 <= agent.age <= 18 else False
    
    def split(self, agent):
        return agent.age

    def weight(self, agent):
        if agent.infection_status == "symptomatic":
            return 0
        else:
            return 6
        
    def project_weights(self, agent1, agent2):
        return min(self.get_weight(agent1), self.get_weight(agent2))

class Ring(popy.Location):
    def subsplit(self, agent):
        pos = self.model.agents.index(agent)
        right = (pos + 1) % len(self.model.agents)
        return [pos, right]

In [ ]:
import datetime as dt

class InfectionModel(popy.Model):
    def setup(self):
        self.creator = popy.Creator(model=self)
        self.agents = self.creator.create_agents(df=df_soep, agent_class=Agent)
        self.locations = self.creator.create_locations(
            agents=self.agents, 
            location_classes=[
                Home, 
                Work, 
                School, 
                Ring,
                ])
        self.date = dt.date(2022, 1, 1)

        for agent in random.choices(self.agents, k=10):
            agent.infection_status = "exposed"
    
    def step(self):
        self.agents.update_infection_status()
        self.agents.infect()

    def update(self):
        self.record(
            "cumulative_infections",
            sum([1 for agent in self.agents if agent.infection_status != "susceptible"]),
        )

In [ ]:
class Agent(popy.Agent):
    def __init__(self, model, *args, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.infection_status = "susceptible"
        self.days_since_infection = 0

    def infect(self):
        if self.infection_status in ["infectious", "symptomatic"]:
            for agent_v in self.neighbors():
                if agent_v.infection_status == "susceptible":
                    contact_weight = self.contact_weight(agent_v)
                    infection_probability = 0.01 * contact_weight

                    if random.random() < infection_probability:
                        agent_v.infection_status = "exposed"


    def update_infection_status(self):
        if self.infection_status in ["exposed", "infectious", "symptomatic"]:
            self.days_since_infection += 1

            if 1 <= self.days_since_infection <= 2:
                self.infection_status = "infectious"

            elif 3 <= self.days_since_infection <= 10:
                self.infection_status = "symptomatic"

            elif self.days_since_infection >= 11:
                self.infection_status = "recovered"

In [ ]:
parameters = {
    "steps": 50,
}
model = InfectionModel(parameters=parameters)
results = model.run()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results.variables.InfectionModel.cumulative_infections)
plt.grid()